In [6]:
#added to use driver
import csv
import numpy as np
from neo4j import GraphDatabase
import pandas as pd
import operator
import re
import math
import random
from random import randint
import numpy as np
import argparse
import torch
import torch.autograd as autograd
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from LSTMTagger import LSTMTagger
from LSTMTrain import LSTMTrain
from LSTMEvaluation import LSTMEvaluation
from datetime import datetime

In [7]:
aux_file = r'rnndata\auxilary-mapping.txt'
rating_file = r'rnndata\rating.txt'
train_file = r'rnndata/training.txt'
test_file = r'rnndata/test.txt'
negative_file = r'rnndata/negative.txt'
pre_train_user_embedding = r'emb/node2vecuser.emb'
pre_train_movie_embedding = r'emb/node2vecmov.emb'
results_file = r'rnndata/results.txt'
positive_path = r'rnndata/positive-path.txt'
negative_path = r'rnndata/negative-path.txt'

In [2]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "a"))

In [10]:
#no need to run again as  I have exported embedidnga nd model 
import networkx as nx
from node2vec import Node2Vec
def createembedding(file, query):
    print('Create the edge list')
    with driver.session() as session, open(file, "w") as edges_file:
        result = session.run(query)

        writer = csv.writer(edges_file, delimiter=" ")

        for row in result:
            writer.writerow([row["source"], row["target"]])
    
def createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME):   
    # Create a graph
    graph=nx.read_edgelist(file, create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])

    # Precompute probabilities and generate walks
    node2vec = Node2Vec(graph, dimensions=10, walk_length=3, num_walks=20, workers=4)

    ## if d_graph is big enough to fit in the memory, pass temp_folder which has enough disk space
    # Note: It will trigger "sharedmem" in Parallel, which will be slow on smaller graphs
    #node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")
    # Embed
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

    # Look for most similar nodes
    #model.wv.most_similar('2')  # Output node names are always strings

    # Save embeddings for later use
    model.wv.save_word2vec_format(EMBEDDING_FILENAME)

    # Save model for later use
    model.save(EMBEDDING_MODEL_FILENAME)

In [11]:
#FILES
file = 'graph/movies.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecmov.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecmov.model'
movienode2vecquery = """\
        MATCH (m:Movie)--(other)
        RETURN id(m) AS source, id(other) AS target
        """

createembedding(file, movienode2vecquery)

Create the edge list


In [12]:
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 9980/9980 [00:00<00:00, 10788.50it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
file = 'graph/users.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecuser.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecuser.model'
usernode2vecquery = """\
        MATCH (u:User)--(other)
        RETURN id(u) AS source, id(other) AS target
        """
createembedding(file, usernode2vecquery)

Create the edge list


In [98]:
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 6257/6257 [00:00<00:00, 26249.72it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [61]:
#This is used to map the auxiliary information (genre) into mapping ID for MovieLens
genrefile = """\
        MATCH (m:Movie)<-[:IS_GENRE_OF]-(g:Genre)
        RETURN id(m), collect(id(g))
        """
with driver.session() as session:
    result = session.run(genrefile)
df = pd.DataFrame([dict(row) for row in result])
df.to_csv(aux_file, header=None, index=None, sep='|', mode='a')
string = open(aux_file).read()
new_str = re.sub('[\\[\\]]', '', string)
open(aux_file, 'w').write(new_str)

120828

In [62]:
ratingfile = """\
        MATCH (u:User)-[r:RATED]->(m:Movie)
        RETURN id(u), id(m), r.rating, r.timestamp
        """
with driver.session() as session:
    result = session.run(ratingfile)
df = pd.DataFrame([dict(row) for row in result])
df.to_csv(rating_file, header=None, index=None, sep='\t', mode='a')

In [63]:
#This is used to split the user-movie interaction data into traning and test according to the timestamp
#load the user-item rating data with the timestamp
def load_data_rating(rating_file):
    fr_rating = open(rating_file,'r')
    rating_data = {}
    for line in fr_rating:
        lines = line.split('\t')
        
        user = lines[0]
        item = lines[1]
        time = lines[3].replace('\n', '')
    
        item_list = []

        if user in rating_data:
            rating_data[user].update({item:time})            
        else:
            rating_data.update({user:{item:time}})
    return rating_data

#split rating_rating data into training and test data by timestamp    
def split_rating_into_train_test(rating_data, train_file, test_file, ratio):  
    fw_train = open(train_file,'w')
    fw_test = open(test_file,'w')
    for user in rating_data:
        item_list = rating_data[user]

        sorted_u = sorted(item_list.items(), key=operator.itemgetter(1))
        sorted_u = dict(sorted_u)
        
        rating_num = rating_data[user].__len__()
        train_size = int(round(rating_num*ratio, 0))# round_int (rating_num, ratio)

        flag = 0

        for item in sorted_u:

            if flag < train_size:
                line = user+'\t'+ item + '\t' + sorted_u[item]+'\n'
                fw_train.write(line)
                flag = flag + 1
            else:
                line = user+'\t' + item + '\t' +sorted_u[item]+'\n'
                fw_test.write(line)

In [64]:
rating_data = load_data_rating(rating_file)
split_rating_into_train_test(rating_data, train_file, test_file, 0.8)

In [65]:
#This is used to sample negative movies that user has not interactions with, so as to balance model training process
#load training data
def load_data_train(file):
    train_dict = {}
    all_movie_list = []

    for line in file:
        lines = line.split('\t')
        user = lines[0]
        movie = lines[1].replace('\n','')
        
        if user not in train_dict:
            init_movie_list = []
            init_movie_list.append(movie)
            train_dict.update({user:init_movie_list})
        else:
            train_dict[user].append(movie)
        
        if movie not in all_movie_list:
            all_movie_list.append(movie)

    return train_dict, all_movie_list

#sample negative movies for all users in training data
def negative_sample(train_dict, all_movie_list, shrink, fw_negative):
    all_movie_size = len(all_movie_list)

    for user in train_dict:
        user_train_movie = train_dict[user]
        user_train_movie_size = len(user_train_movie)
        negative_size = math.ceil(user_train_movie_size * shrink)
        user_negative_movie = []

        while (len(user_negative_movie) < negative_size):
            negative_index = randint(0, (all_movie_size - 1))
            negative_movie = str(all_movie_list[negative_index])
            if negative_movie not in user_train_movie and negative_movie not in user_negative_movie:
                user_negative_movie.append(negative_movie)
                line = user + '\t' + negative_movie + '\n'
                fw_negative.write(line)

In [67]:
fr_train = open(train_file,'r')
fw_negative = open(negative_file,'w')
train_dict, all_movie_list = load_data_train(fr_train)
negative_sample(train_dict, all_movie_list, 0.05, fw_negative)

In [161]:
#Build knowledge graph and mine the connected paths between users and movies in the training data of MovieLens
#load training (positive) or negative user-movie interaction data
def load_data(file):
    data = []
    #print(file)
    for line in file:
        lines = line.split('\t')
        user = lines[0]
        movie = lines[1].replace('\n','')
        data.append((user, movie))

    return data

#add user-movie interaction data into the graph
def add_user_movie_interaction_into_graph(positive_rating):
    Graph = nx.DiGraph()       

    for pair in positive_rating:
        user = pair[0]
        movie = pair[1]
        user_node = 'u' + user
        movie_node = 'i' + movie
        Graph.add_node(user_node)
        Graph.add_node(movie_node)
        Graph.add_edge(user_node, movie_node)

    return Graph



In [167]:
#add auxiliary information (e.g., actor, director, genre) into graph
def add_auxiliary_into_graph(fr_auxiliary, Graph):
    for line in fr_auxiliary:
        lines = line.replace('\n', '').split('|')
        if len(lines) != 2: continue
        
        movie_id = lines[0]
        genre_list = lines[1].split(',')
        #director_list = lines[2].split(',')
        #actor_list = lines[3].split(',')

        #add movie nodes into Graph, in case the movie is not included in the training data
        movie_node = 'i' + movie_id
        if not Graph.has_node(movie_node):
            Graph.add_node(movie_node)

        #add the genre nodes into the graph;  
        #as genre connection is too dense, we add one genre to avoid over-emphasizing its effect
        genre_id = genre_list[0]
        genre_node = 'g' + genre_id
        if not Graph.has_node(genre_node):
            Graph.add_node(genre_node)
        Graph.add_edge(movie_node, genre_node)
        Graph.add_edge(genre_node, movie_node)

        #add the director nodes into the graph
        '''for director_id in director_list:
            director_node = 'd' + director_id
            if not Graph.has_node(director_node):
                Graph.add_node(director_node)
            Graph.add_edge(movie_node, director_node)
            Graph.add_edge(director_node, movie_node)

        #add the actor nodes into the graph
        for actor_id in actor_list:
            actor_node = 'a' + actor_id
            if not Graph.has_node(actor_node):
                Graph.add_node(actor_node)
            Graph.add_edge(movie_node, actor_node)
            Graph.add_edge(actor_node, movie_node)'''

    return Graph

In [172]:
#mine qualified paths between user and movie nodes, and get sampled paths between nodes
def mine_paths_between_nodes(Graph, user_node, movie_node, maxLen, sample_size, fw_file):
    connected_path = [] 
    for path in nx.all_simple_paths(Graph, source=user_node, target=movie_node, cutoff=maxLen):
        #if len(path) >= 4:
             #print(len(path))
        if len(path) == maxLen + 1: 
            connected_path.append(path)

    path_size = len(connected_path)
   
    #as there is a huge number of paths connected user-movie nodes, we get randomly sampled paths
    #random sample can better balance the data distribution and model complexity
    if path_size > sample_size:
        random.shuffle(connected_path)
        connected_path = connected_path[:sample_size]
    
    for path in connected_path:
        #print(path)
        line = ",".join(path) + '\n'
        fw_file.write(line)
        


In [169]:
#dump the postive or negative paths 
def dump_paths(Graph, rating_pair, maxLen, sample_size, fw_file):
    for pair in rating_pair:
        #print(pair)
        user_id = pair[0]
        movie_id = pair[1]
        user_node = 'u' + user_id
        movie_node = 'i' + movie_id

        if Graph.has_node(user_node) and Graph.has_node(movie_node):
            mine_paths_between_nodes(Graph, user_node, movie_node, maxLen, sample_size, fw_file)

In [170]:
fr_training = open(train_file,'r')
fr_negative = open(negative_file, 'r')
fr_auxiliary = open(aux_file,'r')
fw_positive_path = open(positive_path, 'w')
fw_negative_path = open(negative_path, 'w')
positive_rating = load_data(fr_training)
negative_rating = load_data(fr_negative)
print('The number of user-movie interaction data is:  ' + str(len(positive_rating))+ ' \n')
print('The number of negative sampled data is:  ' + str(len(negative_rating))+ ' \n') 

Graph = add_user_movie_interaction_into_graph(positive_rating)
Graph = add_auxiliary_into_graph(fr_auxiliary, Graph)
print('The knowledge graph has been built completely \n')
print('The number of nodes is:  ' + str(len(Graph.nodes()))+ ' \n') 
print('The number of edges is  ' + str(len(Graph.edges()))+ ' \n')

The number of user-movie interaction data is:  28642 

The number of negative sampled data is:  1508 

The knowledge graph has been built completely 

The number of nodes is:  9969 

The number of edges is  48050 



In [176]:
dump_paths(Graph, positive_rating, 3, 5, fw_positive_path)
dump_paths(Graph, negative_rating, 3, 5, fw_negative_path)

In [8]:
input_dim=10
hidden_dim=16
out_dim=1
learning_rate=0.2

fr_postive = open(positive_path, 'r')
fr_negative = open(negative_path, 'r')
fr_pre_user = open(pre_train_user_embedding, 'r')
fr_pre_movie = open(pre_train_movie_embedding, 'r')
fr_train = open(train_file,'r')
fr_test = open(test_file,'r')


node_count = 0 #count the number of all entities (user, movie and attributes)
all_variables = {} #save variable and corresponding id
paths_between_pairs = {} #save all the paths (both positive and negative) between a user-movie pair
positive_label = [] #save the positive user-movie pairs
all_user = [] #save all the users
all_movie = [] #save all the movies

In [9]:
#This is part is aims to feed connected paths into the recurrent neural network to train and test the proposed methods.
#load postive or negative paths, map all nodes in paths into ids
all_variables, paths_between_pairs, positive_label, all_user, all_movie
def load_paths(fr_file, isPositive):    
    node_count = 0
    for line in fr_file:
        #print(line)
        line = line.replace('\n', '')
        lines = line.split(',')
        user = lines[0]
        movie = lines[-1]

        if user not in all_user:
            all_user.append(user)
        if movie not in all_movie:
            all_movie.append(movie)

        key = (user, movie)
        value = []
        path = []

        if isPositive:
            if key not in positive_label:
                positive_label.append(key)

        for node in lines:
            if node not in all_variables:
                all_variables.update({node:node_count})
                node_count = node_count + 1
            path.append(node)

        if key not in paths_between_pairs:
            value.append(path)
            paths_between_pairs.update({key:value})
        else:
            paths_between_pairs[key].append(path)

In [10]:
#load pre-train-user or movie embeddings
global pre_embedding
def load_pre_embedding(fr_pre_file, isUser):
    count = 0
    for line in fr_pre_file:
        if count != 0:            
            lines = line.split(' ')
            node = lines[0]
            if isUser:
                node = 'u' + node
            else:
                node = 'i' + node

            if node in all_variables:
                node_id = all_variables[node]
                embedding = [float(x) for x in lines[1:]]
                embedding = np.array(embedding)
                pre_embedding[node_id] = embedding
        count+=1

In [11]:
#load training or test data
def load_data(fr_file):
    data_dict = {}

    for line in fr_file:
            lines = line.replace('\n', '').split('\t')
            user = 'u' + lines[0]
            item = 'i' + lines[1]

            if user not in data_dict:
                data_dict.update({user:[item]})
            elif item not in data_dict[user]:
                data_dict[user].append(item)

    return data_dict

In [12]:
start_time = datetime.now()
load_paths(fr_postive, True)
load_paths(fr_negative, False)

In [13]:
print ('The number of all variables is :' + str(len(all_variables)))
end_time = datetime.now()
duration = end_time - start_time
print ('the duration for loading user path is ' + str(duration) + '\n')

The number of all variables is :5277
the duration for loading user path is 0:01:50.003111



In [14]:
start_time = datetime.now()
node_size = len(all_variables)
pre_embedding = np.random.rand(node_size, input_dim) #embeddings for all nodes
load_pre_embedding(fr_pre_user, True)
load_pre_embedding(fr_pre_movie, False)
pre_embedding = torch.FloatTensor(pre_embedding)
end_time = datetime.now()
duration = end_time - start_time
print ('the duration for loading embedding is ' + str(duration) + '\n')

the duration for loading embedding is 0:00:00.040874



In [15]:
iteration=5

In [16]:
start_time = datetime.now()
model = LSTMTagger(node_size, input_dim, hidden_dim, out_dim, pre_embedding)
if torch.cuda.is_available():
    model = model.cuda()

#model = model.cpu()
model_train = LSTMTrain(model, iteration, learning_rate, paths_between_pairs, positive_label, \
    all_variables, all_user, all_movie)


C:\Users\meetn\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [17]:
#embedding_dict = model_train.train()
all_user1, all_movie1, model_n = model_train.train()

C:\Users\meetn\Anaconda3\lib\site-packages\torch\nn\functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\meetn\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch[0]: loss is 17.291486152507787
epoch[1]: loss is 63.668797023387015
epoch[2]: loss is 65.12790837045941
epoch[3]: loss is 66.74145469803989
epoch[4]: loss is 64.6142257083593


In [18]:
torch.save(model_n, 'rnnmodel.pt')

C:\Users\meetn\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTMTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\meetn\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\meetn\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\meetn\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
#model_n = torch.load('rnnmodel.pt')

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
#dump the post-train user or item embedding
def dump_post_embedding(all_user1, all_movie1, model_n):
    embedding_dict = {}
    
    node_list = all_user1 + all_movie1
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    #ur_id = self.model
    for node in node_list:
        node_id = torch.LongTensor([int(all_variables[node])])
        node_id = Variable(node_id)
        node_id = node_id.to(device)
        #input_ids_tensor = input_ids_tensor.to(self.device)
        
        node_embedding = model_n.embedding(node_id).squeeze().cpu().data.numpy()
        if node not in embedding_dict:
            embedding_dict.update({node:node_embedding})

    return embedding_dict

In [22]:
model_n = model_n.to(device)
embedding_dict = dump_post_embedding(all_user1, all_movie1, model_n)

In [23]:
print('model training finished')
end_time = datetime.now()
duration = end_time - start_time
print ('the duration for model training is ' + str(duration) + '\n')

model training finished
the duration for model training is 0:30:00.201496



In [24]:
start_time = datetime.now()
train_dict = load_data(fr_train)
test_dict = load_data(fr_test)
model_evaluation = LSTMEvaluation(embedding_dict, all_movie, train_dict, test_dict)
top_score_dict = model_evaluation.calculate_ranking_score()

In [25]:
#print(top_score_dict)

In [26]:
if top_score_dict:
    precision_1,_ = model_evaluation.calculate_results(top_score_dict, 1)
    precision_5,_ = model_evaluation.calculate_results(top_score_dict, 5)
    precision_10, mrr_10 = model_evaluation.calculate_results(top_score_dict, 10)
    end_time = datetime.now()
    duration = end_time - start_time
    print ('the duration for model evaluation is ' + str(duration) + '\n')

    #write_results(fw_results, precision_1, precision_5, precision_10, mrr_10)
    fw_results = open(results_file, 'w')
    line = 'precision@1: ' + str(precision_1) + '\n' + 'precision@5: ' + str(precision_5) + '\n' \
        + 'precision@10: ' + str(precision_10) + '\n' + 'mrr: ' + str(mrr_10) + '\n'
    fw_results.write(line)

    end_time = datetime.now()
    duration = end_time - start_time
    print ('the duration for loading item embedding is ' + str(duration) + '\n')

precision@1 is: 0.008264462809917356
precision@5 is: 0.007438016528925619
precision@10 is: 0.006611570247933886
mrr@10 is: 0.022405876951331498
the duration for model evaluation is 0:00:03.370471

the duration for loading item embedding is 0:00:03.372466

